In [1]:
import numpy as np
import pandas as pd
import os

PATH_TO_REPO = "C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/DataLit-InsideAirbnb"
RAW_DATA_DIR = PATH_TO_REPO + '/data'
SAVING_DIR = PATH_TO_REPO + '/data/preprocessed_data'
CITY_LIST   = ["berlin"]
DEBUG_MODE = True # determines if preprocessing is in DEBUG_MODE (no processing of file --> execution of main-function)

In [23]:
def preprocessing_city_csv_files(city_list):
    """ 
    Converts raw data in RAW_DATA_DIR to proper CSV file format for cities specified in CITY_LIST (see above for global settings).
    Converted files are saved in SAVING_DIR.
    
    """
    print("initializing preprocessing")
    cities_in_raw_data_dir = os.listdir(RAW_DATA_DIR)

    if not set(CITY_LIST).issubset(cities_in_raw_data_dir):
        raise ValueError("not all requested citys are in directory")
    
    data_dict = {}
    
    for city in CITY_LIST:
        data_dict[city] = {}
        city_dir = RAW_DATA_DIR + '/' + city
        data_versions = os.listdir(RAW_DATA_DIR + '/' + city)
        
        for data_version in data_versions:
            data_dict[city][data_version] = {}


            data_version_dir = city_dir + '/' + data_version

            print(os.listdir(data_version_dir))

            FILE_NAMES = [f for f in os.listdir(data_version_dir) if os.path.isfile(os.path.join(data_version_dir, f))]
            
            

            for file_name in FILE_NAMES:
                if file_name.endswith('.csv') or file_name.endswith('.geojson') or file_name.endswith('.csv.gz'):
                    file_path = os.path.join(data_version_dir, file_name)
            
                    # Read the file into a DataFrame
                    if file_name.endswith('.geojson'):
                        df = pd.read_json(file_path)  # Adjust based on the specific geojson handling
                    else:
                        df = pd.read_csv(file_path, index_col=0)

                    basename = file_name.split(sep=".")[0]
                    
                    data_dict[city][data_version][basename] = df
    
    return data_dict

                
    print("preprocessing done")

In [24]:
data_dict = preprocessing_city_csv_files(CITY_LIST)

initializing preprocessing
['calendar.csv', 'listings.csv', 'neighbourhoods.csv', 'reviews.csv', 'summary_information']


c:\Users\nilsk\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


['calendar.csv', 'listings.csv', 'neighbourhoods.csv', 'reviews.csv', 'summary_information']
['calendar.csv', 'listings.csv', 'neighbourhoods.csv', 'reviews.csv', 'summary_information']
['calendar.csv', 'listings.csv', 'neighbourhoods.csv', 'reviews.csv', 'summary_information']


In [75]:
brln_v1_listings = data_dict["berlin"]["18_12_2023"]["listings"]
brln_v2_listings = data_dict["berlin"]["24_03_2024"]["listings"]
brln_v3_listings = data_dict["berlin"]["22_06_2024"]["listings"]
brln_v4_listings = data_dict["berlin"]["18_09_2024"]["listings"]

v1_n = len(brln_v1_listings)
v2_n = len(brln_v2_listings)
v3_n = len(brln_v3_listings)
v4_n = len(brln_v4_listings)

brln_v1_index = set(brln_v1_listings.index.to_list())
brln_v2_index = set(brln_v2_listings.index.to_list())
brln_v3_index = set(brln_v3_listings.index.to_list())
brln_v4_index = set(brln_v4_listings.index.to_list())

print(v1_n)
print(v2_n)
print(v3_n)
print(v4_n)
print()
mutual_indices = brln_v1_index.intersection(brln_v2_index).intersection(brln_v3_index).intersection(brln_v4_index)
one_mutual_index = list(mutual_indices)[346]
print(one_mutual_index)
column = "accommodates"
print(brln_v1_listings.loc[one_mutual_index][column])
print(brln_v2_listings.loc[one_mutual_index][column])
print(brln_v3_listings.loc[one_mutual_index][column])
print(brln_v4_listings.loc[one_mutual_index][column])
print(brln_v4_listings.keys())
print(len(brln_v1_index.difference(brln_v4_index)))
print(len(brln_v4_index.difference(brln_v1_index)))
# print(brln_v1_listings.keys())


13327
13362
13759
14017

4195365
3
3
3
3
Index(['listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
 